# Implementing Digital Control Systems and Simulating Inverted Penedulum


### References and uses

umich stuff
1) https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=SystemModeling -> umich system model
2) https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=SimulinkModeling ->  umich building the model in simulink

3) heres another system model https://www.cantorsparadise.com/modelling-and-simulation-of-inverted-pendulum-5ac423fed8ac

4) heres another hardware / simulation project with some really good sims https://create.arduino.cc/projecthub/zjor/inverted-pendulum-on-a-cart-199d6f


2) https://www.bartleby.com/solution-answer/chapter-10-problem-18pe-college-physics-1st-edition/9781938168000/starting-with-the-formula-for-the-moment-of-inertia-of-a-rod-rotated-around-an-axis-through-one-end/f7039645-7ded-11e9-8385-02ee952b546e -> moment of inertia for the rod

6) state space representation https://en.wikipedia.org/wiki/State-space_representation

creds to tom

todo: 

1) change system so that the equilibrium is at the top and that is theta equals 0

In [20]:
from vpython import*
import math
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statistics import mean
sns.set_theme()
%matplotlib inline
scene = canvas()
scene.background=color.white

class plant_inputs:
    # all units are in SI or derived from SI
    # kg, second, Newtons, meters, etc. 
    
    cart_mass = 0.5 #kg
    cart_starting_pos_x = -1
    cart_coeff_friction = 0.1 #N/m/sec
    rod_mass = 0.2 #kg
    rod_length = 0.7 #meters
    rod_inertia = rod_mass*(rod_length**2)/3
    rod_starting_angle = math.pi/6
    
    
# only the visualization, all dyanmics will be done elsewhere and then passed in values to update the visualization
class plant_visual:
    
    # cart_pos_x is starting x pos, y and z are both zero.
    #rod_angle = float in degrees from -y axis, i.e rod_angle = 180 means rod is straight up
    def __init__(self, cart_pos_x, rod_angle, rod_length):
        
        self.cart_pos_x_init = cart_pos_x
        self.rod_angle_init = rod_angle
        
        box_center = vector(cart_pos_x,0,0)
        
        #pos is defined at the center of the box. 
        self.cart = box(pos = box_center, length = 0.5, width = 0.3, height = 0.2, color = color.blue)
        
        #some parameters for calculating the wheels
        percent_length_between_wheels = 0.6
        wheel_depth = 0.08
        wheel_radius = 0.07
        
        #calculate some parameters for wheel placement
        b1_half =  self.cart.length*percent_length_between_wheels/2
        h1 = self.cart.height/2
        w1 = self.cart.width/2

        #calculate all wheel posiitons
        frwheel_pos = box_center + vector(b1_half,-h1, w1)
        flwheel_pos = box_center + vector(-b1_half,-h1, w1)
        brwheel_pos = box_center + vector(b1_half,-h1, -w1)
        blwheel_pos = box_center + vector(-b1_half,-h1, -w1)
        
        #create wheels
        self.frwheel = cylinder(pos=frwheel_pos, axis = vector(0,0,-wheel_depth), color = color.black, radius = wheel_radius)
        self.flwheel = cylinder(pos=flwheel_pos, axis = vector(0,0,-wheel_depth), color = color.black, radius = wheel_radius)
        self.brwheel = cylinder(pos=brwheel_pos, axis = vector(0,0,wheel_depth), color = color.black, radius = wheel_radius)
        self.blwheel = cylinder(pos=blwheel_pos, axis = vector(0,0,wheel_depth), color = color.black, radius = wheel_radius)
        
        #create center axis for penedulum
        axel_radius = wheel_radius-0.03
        self.axel = cylinder(pos=box_center + vector(0,h1,-w1), axis = vector(0,0,self.cart.width), 
                             color = color.orange, radius = axel_radius)
        
        self.cart.width -= 0.01 #to prevent weird clipping effect. 
        
    
        # NEED TO VERIFY THIS
        # for now putting the penedulum at the top of the cart.not sure if this changes the dynamics or not
        # will need to verify
        # https://www.glowscript.org/docs/VPythonDocs/rotation.html
        # there are functions for converting back and forth from radians to degrees
        
        # define the penedulum rod.
        self.rod = cylinder(pos=box_center + vector(0,h1,0), axis = vector(0,-rod_length,0), 
                            radius = axel_radius, color = color.red)
        self.rod.angle = 0
        
        #merge everything but the rod? or do i want to include the rod.
        self.cartsys = compound([self.cart, self.frwheel, self.flwheel, self.brwheel, self.blwheel, self.axel])
        

    def update_pos(self, abs_pos_x):
        self.cartsys.pos.x = abs_pos_x
        self.rod.pos.x = abs_pos_x
        
    def update_rod(self, abs_ang):
        self.rod.rotate(angle = abs_ang-self.rod.angle, axis = self.axel.axis)
        self.rod.angle = abs_ang
        
    def test_plant_motion(self):
        
        pos = self.cart_pos_x_init
        angle = self.rod_angle_init
        
        for i in range(0,200):
            rate(20)
            pos += 0.02
            angle+=4
            self.update_pos(pos)
            self.update_rod(angle)

        for i in range(0,200):
            rate(20)
            pos -= 0.02
            angle-=4
            self.update_pos(pos)
            self.update_rod(angle)

        
class plant_dynamics:
    #USE NUMPY ARRAYS CAUSE LINAG STUFF IS BUILT INTO THOSE, and they more efficient.
    # M = mass of cart
    # m = mass of pendulum
    # b = coeff of friction for cart
    # l = length to penedulm center of mass THE FORCE ACTS ON THE CENTER OF MASS MAKE SURE YOU DIVIDE BY TWO
    # I = mass moment of intertia of the pendulum
    # F = force applied to the cart, IS THIS FORCE THE INPUT TO THE SYSTEM?? or is it just the position x??
    # x = cart position coordinate
    # theta = pendulum angle
    
    # all the parameters will be passed in
    def __init__(self, cart_mass, rod_mass, coeff_friction_cart, rod_length, rod_inertia, 
                 cart_pos_x_init, rod_angle_init, force_init):
        
        # state-space representation
        # x' = Ax + Bu
        # y = Cx + Du
        
        self.M = cart_mass
        self.m = rod_mass
        self.b = coeff_friction_cart
        self.l_2 = rod_length/2 #length to the center of the rod
        self.I = rod_inertia
        self.g = 9.8 #m/s
        
        denom = self.I*(self.M + self.m) + self.M*self.m*self.l_2**2
        
        #define numpy arrays in [row, col] format
        # state matrix
        self.A = np.array([[0, 1, 0, 0],
                           [0, -1*(self.I + self.m*self.l_2**2)*self.b/denom, (self.m**2)* self.g * (self.l_2**2)/denom, 0],
                           [0, 0, 0, 1],
                           [0, -1*self.m*self.l_2*self.b/denom, self.m*self.g*self.l_2*(self.M+self.m)/denom, 0]],
                           dtype = float)
        
        #input matrix
        self.B = np.array([[0],
                           [(self.I+self.m*self.l_2**2)/denom],
                           [0],
                           [self.m*self.l_2/denom]],
                           dtype = float)
        
        #ouput matrix
        self.C = np.array([[1,0,0,0],
                           [0,0,1,0]], 
                           dtype = float)
        
        #feedforward matrix, in our case zero matrix
        self.D = np.array([[0],
                           [0]], 
                           dtype = float)
        
        #state vector
        #YOU MAY NEED TO TWEAK THIS BASED ON INITIAL CONDITIONS
        self.x = np.array([[cart_pos_x_init], [0], [rod_angle_init], [0]], dtype = float)
        
        #derivative of state vector
        self.x_next_state = np.array([[0],[0],[0],[0]], dtype= float)
        
        #output vector
        self.y = np.array([[0],[0]], dtype = float)
        
        #input/control vector
        self.u = force_init
        
        #identity matrix
        self.I = np.array([[1,0,0,0], [0,1,0,0], [0,0,1,0],[0,0,0,1]])
        
        
        
        
    def solve_system(self, dt, new_force):


        # self.x_dot = self.A @ self.x + self.B*self.u 
        # self.y = self.C @ self.x + self.D*self.u 
        
        self.u = new_force
        # https://en.wikipedia.org/wiki/Discretization#:~:text=In%20applied%20mathematics%2C%20discretization%20is,and%20implementation%20on%20digital%20computers.
        self.x_next_state = np.linalg.inv(self.I - self.A*dt) @ self.x + (self.B*dt)*self.u
        self.y = self.C @ self.x + self.D*self.u 
        
        
        
    def update_matrices(self):
        
        # copy x[t+dt] to x[t] for the next run
        self.x = np.copy(self.x_next_state)
        
        #first element is cart pos and 2nd angle is rod angle from -y axis
        return [self.y[0,0], self.y[1,0]]
        
    
#         self.x[1,0] = self.x_dot[0,0]
#         self.x[3,0] = self.x_dot[2,0]
        
#         #x(t+dt) = x(t) + v(t + dt)*dt
#         self.x[0,0] = self.x[0,0] + self.x_dot[1,0]* dt
        
#         #0(t+dt) = 0(t) + w(t + dt)*dt
#         self.x[2,0] = self.x[2,0] + self.x_dot[3,0]* dt

#         a_cart =  self.x_dot[1,0]
#         v_cart = self.x[1,0] + a_cart*dt
#         x_cart = self.x[0,0] + v_cart*dt
        
#         a_rod =  self.x_dot[3,0]
#         v_rod = self.x[3,0] + a_cart*dt
#         x_rod = self.x[2,0] + v_cart*dt

    
#         self.x[0,0] = x_cart
#         self.x[1,0] = v_cart
#         self.x[2,0] = x_rod
#         self.x[3,0] = v_rod

        
    def show_system(self):
#         print(self.A)
#         print("A^\n-----\n")
        
#         print(self.B)
#         print("B^\n-----\n")
              
#         print(self.C)
#         print("C^\n-----\n")
        
#         print(self.D)
#         print("D^\n-----\n")
        
        print(self.x_next_state)
        print("x_dot^\n-----\n")
        
        print(self.x)
        print("x^\n-----\n")
        
        print(self.y)
        print("y^\n-----\n")
        
        print("NEXT TIME")
        
#         print(self.u)
#         print("u^\n-----\n")
    

class system_inputs:
    # this should be system inputs or something not plant inputs.
    input_force = 0 #0.001 #netwons, force is applied to the cart and is initial disturbance for the system.
        
    
#plot axis
yaxis = arrow(pos=vector(0,-1.5,0), axis=vector(0, 3,0), shaftwidth=0.01, color=color.green, headwidth = 0.02 ) 
xaxis = arrow(pos=vector(-1.5,0,0), axis=vector(3,0,0), shaftwidth=0.01, color=color.red, headwidth = 0.02 )
    
#simulation
plant_motion = plant_visual(plant_inputs.cart_starting_pos_x, plant_inputs.rod_starting_angle, plant_inputs.rod_length)
# plant_motion.test_plant_motion()


# def __init__(self, cart_mass, rod_mass, coeff_friction_cart, rod_length, rod_inertia, 
#              cart_pos_x_init, rod_angle_init, force_init):
plant_dynam = plant_dynamics(plant_inputs.cart_mass, plant_inputs.rod_mass, 
                             plant_inputs.cart_coeff_friction, plant_inputs.rod_length,
                             plant_inputs.rod_inertia, plant_inputs.cart_starting_pos_x, 
                             plant_inputs.rod_starting_angle, system_inputs.input_force)


t = 0
sim_rate = 50
delta_t = 0.001
t_end = 20

def get_force(t, dt):
    pass

# plant_dynam.show_system()
# plant_dynam.solve_system()
# plant_dynam.update_matrices(delta_t, 0.001)
# print("AFTER MATRIX UPDATE")
# plant_dynam.show_system()

# #nextupdate
# plant_dynam.solve_system()
# plant_dynam.update_matrices(delta_t, 0.001)
# print("AFTER MATRIX UPDATE")
# plant_dynam.show_system()

# plant_dynam.show_system()

# g1 = graph(width = 900, scroll = True, fast = inputs.OPTIMIZE_GRAPH) 
# f1 = gdots(graph=g1,color=color.red, radius = 1, interval = 60)
# f2 = gdots(graph=g1,color=color.blue, radius = 1, interval = 60)

# g2 = graph(width = 900, scroll = True, fast = inputs.OPTIMIZE_GRAPH)
# f3 = gcurve(graph=g2,color=color.orange, label="freq")

while t < 4:#delta_t*100:
    rate(sim_rate)
    plant_dynam.solve_system(delta_t, 0)
    [pos,angle] = plant_dynam.update_matrices()
    #     plant_dynam.show_system()
    plant_motion.update_pos(pos)
    plant_motion.update_rod(angle)
    scene.center = plant_motion.cartsys.pos

    #     f1.plot(t, )
    #     f2.plot(,HWR.vin)
    #     f3.plot(HWR.t,HWR.volt_source.freq)

    t+=delta_t
    



    




<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [ ]:
from vpython import*
import math
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statistics import mean
sns.set_theme()
%matplotlib inline
scene = canvas()
scene.background=color.white

class plant_inputs:
    # all units are in SI or derived from SI
    # kg, second, Newtons, meters, etc. 
    
    cart_mass = 0.5 #kg
    cart_starting_pos_x = -1
    cart_coeff_friction = 0.1 #N/m/sec
    rod_mass = 0.2 #kg
    rod_length = 0.7 #meters
    rod_inertia = rod_mass*(rod_length**2)/3
    rod_starting_angle = math.pi
    
    
# only the visualization, all dyanmics will be done elsewhere and then passed in values to update the visualization
class plant_visual:
    
    # cart_pos_x is starting x pos, y and z are both zero.
    #rod_angle = float in degrees from -y axis, i.e rod_angle = 180 means rod is straight up
    def __init__(self, cart_pos_x, rod_angle, rod_length):
        
        self.cart_pos_x_init = cart_pos_x
        self.rod_angle_init = rod_angle
        
        box_center = vector(cart_pos_x,0,0)
        
        #pos is defined at the center of the box. 
        self.cart = box(pos = box_center, length = 0.5, width = 0.3, height = 0.2, color = color.blue)
        
        #some parameters for calculating the wheels
        percent_length_between_wheels = 0.6
        wheel_depth = 0.08
        wheel_radius = 0.07
        
        #calculate some parameters for wheel placement
        b1_half =  self.cart.length*percent_length_between_wheels/2
        h1 = self.cart.height/2
        w1 = self.cart.width/2

        #calculate all wheel posiitons
        frwheel_pos = box_center + vector(b1_half,-h1, w1)
        flwheel_pos = box_center + vector(-b1_half,-h1, w1)
        brwheel_pos = box_center + vector(b1_half,-h1, -w1)
        blwheel_pos = box_center + vector(-b1_half,-h1, -w1)
        
        #create wheels
        self.frwheel = cylinder(pos=frwheel_pos, axis = vector(0,0,-wheel_depth), color = color.black, radius = wheel_radius)
        self.flwheel = cylinder(pos=flwheel_pos, axis = vector(0,0,-wheel_depth), color = color.black, radius = wheel_radius)
        self.brwheel = cylinder(pos=brwheel_pos, axis = vector(0,0,wheel_depth), color = color.black, radius = wheel_radius)
        self.blwheel = cylinder(pos=blwheel_pos, axis = vector(0,0,wheel_depth), color = color.black, radius = wheel_radius)
        
        #create center axis for penedulum
        axel_radius = wheel_radius-0.03
        self.axel = cylinder(pos=box_center + vector(0,h1,-w1), axis = vector(0,0,self.cart.width), 
                             color = color.orange, radius = axel_radius)
        
        self.cart.width -= 0.01 #to prevent weird clipping effect. 
        
    
        # NEED TO VERIFY THIS
        # for now putting the penedulum at the top of the cart.not sure if this changes the dynamics or not
        # will need to verify
        # https://www.glowscript.org/docs/VPythonDocs/rotation.html
        # there are functions for converting back and forth from radians to degrees
        
        # define the penedulum rod.
        self.rod = cylinder(pos=box_center + vector(0,h1,0), axis = vector(0,-rod_length,0), 
                            radius = axel_radius, color = color.red)
        self.rod.angle = 0
        
        #merge everything but the rod? or do i want to include the rod.
        self.cartsys = compound([self.cart, self.frwheel, self.flwheel, self.brwheel, self.blwheel, self.axel])
        

    def update_pos(self, abs_pos_x):
        self.cartsys.pos.x = abs_pos_x
        self.rod.pos.x = abs_pos_x
        
    def update_rod(self, abs_ang):
        self.rod.rotate(angle = abs_ang-self.rod.angle, axis = self.axel.axis)
        self.rod.angle = abs_ang
        
    def test_plant_motion(self):
        
        pos = self.cart_pos_x_init
        angle = self.rod_angle_init
        
        for i in range(0,200):
            rate(20)
            pos += 0.02
            angle+=4
            self.update_pos(pos)
            self.update_rod(angle)

        for i in range(0,200):
            rate(20)
            pos -= 0.02
            angle-=4
            self.update_pos(pos)
            self.update_rod(angle)


class plant_dynamics:
    #USE NUMPY ARRAYS CAUSE LINAG STUFF IS BUILT INTO THOSE, and they more efficient.
    # M = mass of cart
    # m = mass of pendulum
    # b = coeff of friction for cart
    # l = length to penedulm center of mass THE FORCE ACTS ON THE CENTER OF MASS MAKE SURE YOU DIVIDE BY TWO
    # I = mass moment of intertia of the pendulum
    # F = force applied to the cart, IS THIS FORCE THE INPUT TO THE SYSTEM?? or is it just the position x??
    # x = cart position coordinate
    # theta = pendulum angle
    
    # all the parameters will be passed in
    def __init__(self, cart_mass, rod_mass, coeff_friction_cart, rod_length, rod_inertia, 
                 cart_pos_x_init, rod_angle_init, force_init):
        
        # state-space representation
        # x' = Ax + Bu
        # y = Cx + Du
        
        self.M = cart_mass
        self.m = rod_mass
        self.b = coeff_friction_cart
        self.l_2 = rod_length/2 #length to the center of the rod
        self.I = rod_inertia
        self.g = 9.8 #m/s
        
    def update(self, force):
        #ouptut of the pid controller will be force
        # so we input it here.
        
        
    
            


#phils lab pid controller on youtube
class PID_controller:
    def __init__(self, kp, ki, kd, T):
        
        self.out = 0
        self.ki = ki
        self.kp = kp
        self.kd = kd
        
        #derivate low-pass filter time const
        self.tau = 300* 15e-6
        
        #limits onthe controller
        self.lim_min = -30
        self.lim_max = 30
        
        #sample time in sec
        self.T = T
        
        self.prev_error = 0
        self.integrator = 0
        self.differen = 0
        self.prev_measure = 0
        
    def update(setpoint, measurement) -> float:
        
        error = setpoint - measurement
        
        #proportional term
        p_term = self.kp*error
        
        #integral term
        self.integrator = self.integrator + 0.5*self.ki * self.T * (error + self.prev_error)
        
        # anti wind up - dynamic integrator clamping
        lim_min_integrator = 0, lim_max_integrator = 0
        
        if(self.lim_max > p_term):
            lim_max_integrator = self.lim_max - p_term
        else:
            lim_max_integrator = 0
            
        if(self.lim_min < p_term):
            lim_min_integrator = self.lim_min - p_term
        else:
            lim_min_integrator = 0
            
        # clamp integrator using the values that were just calculated
        if self.integrator > lim_max_integrator:
            self.integrator = lim_max_integrator
        elif self.integrator < lim_min_integrator:
            self.integrator = lim_min_integrator
            
        # differentiator (band limited one)
        self.differen = (2*self.kd * (measurement - self.prev_measure) 
                        + (2*self.tau - self.T) * self.differen)
                        / (2*self.tau + self.T)
        
        # sum all the terms
        self.out = p_term + self.integrator + self.differen
        
        if self.out > self.lim_max:
            self.out = self.lim_max
        elif self.out < self.lim_min:
            self.out = self.lim_min
            
        self.prev_error = error
        self.prev_measure = measurement
        
        return self.out